A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `YOUR CODE HERE`. Do **not** write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed or overwritten by the autograder.

2. Before you submit your assignment, make sure everything runs as expected. Go to menubar, select _Kernel_, and restart the kernel and run all cells (_Restart & Run all_).

3. Do not change the title (i.e. file name) of this notebook.

4. Make sure that you save your work (in the menubar, select _File_ → _Save and CheckPoint_)

5. You are allowed to submit an assignment multiple times, but only the most recent submission will be graded.

## Problem 1. Bayesian Linear Regression.

In this problem, we will use `pymc3` to estimate the model parameters of a linear regression model that predicts `AirTime` from `Distance`.

![](https://github.com/UI-DataScience/accy570-fa16/raw/master/Week14/images/bayesian_linear_regression.png)

(Compare this plot with the plots in [ACCY571 Week 5 Problem 1](https://github.com/UI-DataScience/accy571-fa16/blob/master/Week5/assignments/Problem_1_Seaborn_Linear_Regression.ipynb) and [Problem 2](https://github.com/UI-DataScience/accy571-fa16/blob/master/Week5/assignments/Problem_2_Statsmodels_Linear_Regression.ipynb).)

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm

from nose.tools import assert_equal, assert_true, assert_is_instance
from numpy.testing import assert_array_almost_equal, assert_almost_equal

Suppose we are given the air times (in minutes) and distances traveled (in miles) of 20 different flights, and we want to find the relationship between the air time and distance by fitting a linear regression model using Bayesian methods. In the following code cell, `data` is given as a Pandas DataFrame.

```python
>>> print(data)
```
```
    AirTime  Distance
0        60       361
1        84       569
2        95       588
3       182      1172
4       337      2565
5       119       861
6        87       665
7       103       787
8        55       228
9        47       197
10      127       978
11      215      1745
12      213      1605
13       59       373
14       31       156
15       57       209
16       88       505
17       42       224
18       45       282
19      102       862
```

(Note: This is the same data we use in [ACCY 571 Week 5](https://github.com/UI-DataScience/accy571-fa16/blob/master/Week5/assignments/Problem_1_Seaborn_Linear_Regression.ipynb).)

In [ ]:
data = pd.DataFrame(
    {"AirTime": [60, 84, 95, 182, 337, 119, 87, 103, 55, 47,
        127, 215, 213, 59, 31, 57, 88, 42, 45, 102],
     "Distance": [361, 569, 588, 1172, 2565, 861, 665, 787, 228, 197,
        978, 1745, 1605, 373, 156, 209, 505, 224, 282, 862]}
)

print(data)

## Use PyMC3 to fit a linear regression model

- Complete the `get_trace()` function which implements the following model using `pymc3`:
$$
\begin{aligned}
Y  &\sim \mathcal{N}(\mu, \sigma^2) \\
\mu &= \alpha + \beta X \\
\alpha &\sim \mathcal{N}(\mu=0, \sigma^2=1) \\
\beta &\sim \mathcal{N}(\mu=10, \sigma^2=1) \\
\sigma &\sim \mathcal{U}(0, 100)
\end{aligned}
$$
where $\mathcal{N}(\mu, \sigma^2)$ denotes the [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) and $\mathcal{U}$ is the <a href="https://en.wikipedia.org/wiki/Uniform_distribution_(continuous)">uniform distribution</a>.

- Use the `pymc3` variable names `alpha`, `beta`, and `sigma` for $\alpha$, $\beta$, and $\sigma$, respectively. (The [Bayesian Modeling notebook](https://github.com/UI-DataScience/accy570-fa16/blob/master/Week14/notebooks/intro2pp-bm.ipynb) uses the names `Intercept`, `Slope`, and `sigma`. Do not use `Intercept` and `Slope`. Be sure to change variable names.)

Hints:

- The bulk of the code is provided. Fill in the spaces between
```python
with pm.Model() as model:
```
and
```python
    y = pm.Normal('y', mu=y_exp, sd=sigma, observed=y)
```

- You basically need to write only 4 lines:
  1. Define `alpha` (the intercept),
  2. Define `beta` (the slope),
  3. Define `sigma` (the standard deviaiton of the posterior distibution $Y$), and
  4. Define `y_exp` (the mean of the posterior distribution).
  
- Refer to the [Bayesian Modeling notebook](https://github.com/UI-DataScience/accy570-fa16/blob/master/Week14/notebooks/intro2pp-bm.ipynb) and [Getting Started with PyMC3](https://pymc-devs.github.io/pymc3/notebooks/getting_started.html).
- `pymc3.find_MAP()` estimates the model paramters with the [maximum a posteriori (MAP) method](https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation).

- `pm.NUTS()` uses the [No-U-Turn Sampler (NUTS)](https://arxiv.org/abs/1111.4246) to generate posterior samples.
- Note that for reproducibility we use the `random_seed` parameter in `pymc3.sample()`.

In [ ]:
def get_trace(X, y, n_samples=1000, random_seed=0):
    """
    A simple Bayesian linear regression model with normal priors.
    
    Paramters
    ---------
    X: A numpy array
    y: A numpy array
    n_samples: The number of samples to draw in pymc3.sample().
               Defaults to 1000.
    random_seed: An int. Used in pymc3.sample().
                 Defaults to 0.
                 
    Returns
    -------
    A pymc3.MultiTrace object with access to sampling values.
    """
    
    with pm.Model() as model:
        
        # YOUR CODE HERE

        y = pm.Normal('y', mu=y_exp, sd=sigma, observed=y)
        
        start = pm.find_MAP()
        step = pm.NUTS(scaling=start)
        trace = pm.sample(n_samples, step=step, start=start, random_seed=random_seed)
        
    return trace

Now that the sampler has been created, we can use it to create a chain of sampled parameter values, which are known as _traces_. After a (large) number of samples have been generated, these traces will provide posterior distributions for the model parameters. Thus, we can use the traces to not only obtain the model parameter values, but a statistical characterization for these values.

The following code cell may take a while to complete. Please be patient.

In [ ]:
trace = get_trace(data.Distance.values, data.AirTime.values, n_samples=1000, random_seed=0)

In [ ]:
assert_is_instance(trace, pm.backends.base.MultiTrace)

assert_true('alpha' in trace.varnames)
assert_true('beta' in trace.varnames)
assert_true('sigma' in trace.varnames)

for v in trace.varnames:
    assert_equal(len(trace[v]), 1000)

assert_almost_equal(trace['alpha'][0], 0.40060267400537569)
assert_almost_equal(trace['beta'][0], 0.13549270335335292)
assert_almost_equal(trace['sigma'][0], 11.635038372477414)

assert_array_almost_equal(
    trace['alpha'][-5:],
    [ 1.552465,  1.573076,  1.571528,  1.571528,  1.374191]
    )
assert_array_almost_equal(
    trace['beta'][-5:],
    [ 0.127634,  0.129961,  0.135609,  0.135609,  0.130797]
    )
assert_array_almost_equal(
    trace['sigma'][-5:],
    [ 16.317306,  16.641373,  16.73387 ,  16.73387 ,  16.804086]
    )

Let's check our model by inspecting the sampling output. A simple posterior plot can be created using `traceplot()`.

In [ ]:
# Plot traces
sns.set_style('darkgrid')
pm.traceplot(trace[-1000:], ['alpha', 'beta', 'sigma'])
plt.show()

Finally, we can visually inspect the fit of our model by displaying the data points and the best fit regression line (along with error estimation), all in a single plot.

In [ ]:
n_samples = 1000

x = data.Distance.values
y = data.AirTime.values

sns.set_style('white')

fig, ax = plt.subplots()

ax.scatter(x, y, c=sns.xkcd_rgb['denim blue'], label = 'Sample Data')

xl = x.min()
xh = x.max()

intercepts = trace['alpha'][-n_samples:]
slopes = trace['beta'][-n_samples:]

for m, b in zip(slopes, intercepts):
    yl = m * xl + b
    yh = m * xh + b
    ax.plot((xl, xh), (yl, yh), color=sns.xkcd_rgb['medium green'], lw=0.1, alpha = 0.1)

# Replot last one to get legend label
ax.plot((xl, xh), (yl, yh), color=sns.xkcd_rgb['medium green'], lw=0.1, label = 'Posterior Prediction')

m_fit = slopes.mean()
b_fit = intercepts.mean()

yfl = b_fit + m_fit * xl
yfh = b_fit + m_fit * xh
ax.plot((xl, xh), (yfl, yfh), color=sns.xkcd_rgb['pale red'], lw=3, label='Model Regression')

ax.set_xlim(0, 2000)
ax.set_ylim(0, 300)

ax.legend(loc='upper left', frameon=True)

ax.set_xlabel('Distance (miles)')
ax.set_ylabel('Air time (min)')

sns.despine(offset=10, trim=True)

plt.show()